In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable
import matplotlib.pyplot as plt

from torch.optim import lr_scheduler

batch_size=64
learning_rate=0.05
num_epoch=50

In [2]:
cifar_train=dset.CIFAR10("CIFAR10/",train=True, transform=transforms.ToTensor(),target_transform=None,download=True)
cifar_test=dset.CIFAR10("CIFAR10/",train=False,transform=transforms.ToTensor(),target_transform=None,download=True)
cifar_train=dset.CIFAR10("CIFAR10/",train=True,
                         transform=transforms.Compose([transforms.ToTensor(),
                                                       transforms.Normalize(mean=(0.5,0.5,0.5),std=(0.5,0.5,0.5)),
                                                       ])
                         , target_transform=None,download=False)
cifar_test=dset.CIFAR10("CIFAR10/",train=False,
                        transform=transforms.Compose([transforms.ToTensor(),
                                                       transforms.Normalize(mean=(0.5,0.5,0.5),std=(0.5,0.5,0.5)),
                                                       ])
                         , target_transform=None,download=False)


Extracting CIFAR10/cifar-10-python.tar.gz to CIFAR10/
Files already downloaded and verified


In [3]:
def ComputeAccr(dloader,imodel):
    correct=0
    total=0
    
    for j,[imgs,labels] in enumerate(dloader):
        img=Variable(imgs,volatile=True).cuda()
        label=Variable(labels).cuda()
        
        output=imodel.forward(img)
        _,output_index=torch.max(output,1)
        
        total+=label.size(0)
        correct+=(output_index==label).sum().float()
    print("Accuracy of Test Data: {}".format(100*correct/total))

In [4]:
train_loader=torch.utils.data.DataLoader(list(cifar_train)[:],batch_size=batch_size,shuffle=True,num_workers=2,drop_last=True)
test_loader=torch.utils.data.DataLoader(cifar_test,batch_size=batch_size,shuffle=False,num_workers=2,drop_last=True)

class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.layer=nn.Sequential(
            nn.Conv2d(3,16,3,padding=1),
            nn.ELU(),
            nn.Dropout2d(0,2),
            nn.BatchNorm2d(16),
            nn.Conv2d(16,32,3,padding=1),
            nn.ELU(),
            nn.Dropout2d(0,2),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(2,2),
            nn.Conv2d(32,64,3,padding=1),
            nn.ELU(),
            nn.Dropout2d(0,2),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(2,2)
        )
        self.fc_layer=nn.Sequential(
            nn.Linear(64*8*8,100),
            nn.ELU(),
            nn.Dropout2d(0,2),
            #nn.BatchNorm1d(100),
            nn.Linear(100,10)
        )
        # for m in self.modules():
        #  if isinstance(m,nn.Conv2d):
        #    init.kaiming_normal(m.weight.data)
        #    m.bias.data.fill_(0)
        #  if isinstance(m,nn.Linear):
        #    init.kaiming_normal(m.weight.data)
        #    m.bias.data.fill_(0)
    def forward(self,x):
        out=self.layer(x)
        out=out.view(batch_size,-1)
        out=self.fc_layer(out)
        
        return out
model=CNN().cuda()

In [5]:
loss_func=nn.CrossEntropyLoss()
optimizer=torch.optim.SGD(model.parameters(), lr=learning_rate)
scheduler=lr_scheduler.StepLR(optimizer,step_size=20,gamma=0.2)
model.train()
for i in range(num_epoch):
    for j,[image,label] in enumerate(train_loader):
        x=Variable(image).cuda()
        y_=Variable(label).cuda()
        optimizer.zero_grad()
        output=model.forward(x)
        loss=loss_func(output,y_)
        loss.backward()
        optimizer.step()
        
        if j%1000==0:
            print(j,loss)

0 tensor(2.2980, device='cuda:0', grad_fn=<NllLossBackward>)
0 tensor(0.8509, device='cuda:0', grad_fn=<NllLossBackward>)
0 tensor(0.6064, device='cuda:0', grad_fn=<NllLossBackward>)
0 tensor(0.4333, device='cuda:0', grad_fn=<NllLossBackward>)
0 tensor(0.3541, device='cuda:0', grad_fn=<NllLossBackward>)
0 tensor(0.3627, device='cuda:0', grad_fn=<NllLossBackward>)
0 tensor(0.2233, device='cuda:0', grad_fn=<NllLossBackward>)
0 tensor(0.0690, device='cuda:0', grad_fn=<NllLossBackward>)
0 tensor(0.0801, device='cuda:0', grad_fn=<NllLossBackward>)
0 tensor(0.0515, device='cuda:0', grad_fn=<NllLossBackward>)
0 tensor(0.0189, device='cuda:0', grad_fn=<NllLossBackward>)
0 tensor(0.0089, device='cuda:0', grad_fn=<NllLossBackward>)
0 tensor(0.0163, device='cuda:0', grad_fn=<NllLossBackward>)
0 tensor(0.0095, device='cuda:0', grad_fn=<NllLossBackward>)
0 tensor(0.0016, device='cuda:0', grad_fn=<NllLossBackward>)
0 tensor(0.0027, device='cuda:0', grad_fn=<NllLossBackward>)
0 tensor(0.0015, device=

In [6]:
model.eval()
ComputeAccr(test_loader,model)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  


Accuracy of Test Data: 77.02323913574219


In [7]:
netname='./test.pkl'
torch.save(model,netname, )